### Importing Necessary Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
from scipy.stats import stats

In [4]:
from sklearn import metrics
import pandas_profiling

In [5]:
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

### Importing Data Set

In [6]:
Data = pd.read_excel("Data Set.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'Data Set.xlsx'

### Setting Dispaly Options so that all the columns can be viewed

In [ ]:
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',1000)
Data

#### 1. Creating target "Total_Spend" variable by combining cardspent + card2spent and later dropping them
#### 2. Removing duplicate categorical and log variable of the numerical variable from the data set
#### 3. Replacing the value "-1" by 0 in spoused and carvalue varibles and null values by 0.0 in continous variable

In [ ]:
Data['Total_Spend'] = Data['cardspent'] + Data['card2spent']
Data = Data.drop(columns={'cardspent',"card2spent"},axis = 1)
Data = Data.drop(columns={"agecat","edcat","empcat","lninc","inccat","lncreddebt","lnothdebt","spousedcat","addresscat","commutecat","cardtenurecat","lnlongmon","lnlongten","lntollmon","lntollten","lnequipmon","lnequipten","lncardmon","lncardten","lnwiremon","lnwireten"},axis = 1)
Data = Data.drop(columns={'custid'},axis=1)
Data = Data.drop(columns={'birthmonth'},axis=1)
Data = Data.drop(columns={'carcatvalue'},axis=1)
Data = Data.drop(columns={'card2tenurecat'},axis=1)

In [ ]:
Data.spoused[Data.spoused==-1]=0
Data.carvalue[Data.carvalue==-1] = 0
Data.cardten[Data.cardten.isna()==True] = 0.0
Data.longten[Data.longten.isna()==True] = 0.0

In [ ]:
Data.townsize = Data.fillna(Data.townsize.mode())

In [ ]:
def missing(x):
    x = x.fillna(x.mean())
    return x

In [ ]:
Data = Data.apply(lambda x: missing(x))

In [ ]:
def outliers(x):
    lc = x.quantile(0.01)
    uc = x.quantile(0.99)
    return x.clip(lower = lc, upper = uc)

In [ ]:
Data= Data.apply(lambda x : outliers(x))

In [ ]:
sns.distplot(Data.Total_Spend)
plt.show()

##### Creating log of the target  varaible as target variable does not follow normal distribution

In [ ]:
Data.Total_Spend = np.log(Data.Total_Spend)
sns.distplot(Data.Total_Spend)
plt.show()

##### Seprating Numerical data from the data set with the help of data dictionary

In [ ]:
Data_num = Data.loc[:,['age','ed',"employ",'income','debtinc','creddebt','othdebt','spoused','reside','pets','pets_cats','pets_dogs','pets_birds','pets_reptiles','pets_small',
                      'pets_saltfish','pets_freshfish','address','cars','carvalue','commutetime','cardtenure','card2tenure','tenure','longmon',
                      'longten','tollmon','tollten','equipmon','equipten','cardmon','cardten','wiremon','wireten','hourstv','Total_Spend']]

In [ ]:
Data_num.head(2)

#### Univariate Analysis of Numerical Variables

In [ ]:
#for i in Data_num.columns:
#    Data_num[i].hist()
#    plt.xlabel(str(i))
#    plt.show()

1. More than 30% have job experience less than 5 years
2. 50% of the customers have income less than equal to 50K
3. 60% customers have credit debt less than 2K
4. Customers genrally have 2 cars

In [ ]:
def var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.std()/x.mean(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                  index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'CV','MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])
Data_num.apply(lambda x: var_summary(x)).T

#### Bivariate Analysis of Numerical Variable

In [ ]:
#for i in Data_num.columns:
#    sns.scatterplot(data=Data_num,x=Data_num[i],y="Total_Spend")
#    plt.show()

1. As income increases their is a increase in the total_spend, creddebt & otherdebt also have same realtion with income
2. Carvalue have a positive corealtion with total_spend

#### Seprating Categorical Features from the data set

In [ ]:
Data_cat = Data.loc[:,(Data.columns.difference(['age','ed',"employ",'income','debtinc','creddebt','othdebt','spoused','reside','pets','pets_cats','pets_dogs','pets_birds','pets_reptiles','pets_small',
                      'pets_saltfish','pets_freshfish','address','cars','carvalue','commutetime','cardtenure','card2tenure','tenure','longmon',
                      'longten','tollmon','tollten','equipmon','equipten','cardmon','cardten','wiremon','wireten','hourstv','Total_Spend']))]

#### Univariate Analysis of Categorical Features

In [ ]:
#for i in Data_cat.columns:
#    Data_cat[i].hist()
#    plt.xlabel(str(i))
#    plt.show()

## Univariate Analysis of Categorical Variable

1. More than 70% of customers uses CallCard Service
2. 60% of the customers have not leased/bought the car in previous years and they are not planning to buy in coming year as well
3. Approx 50% of customers have Discover and Visa as their primary card
4. Nearly 50% customers have Visa and Mastercard as their secondary card
5. Customer tend to prefer have "other & Airline Miles" as benefits in their secondary card
6. 80% customer prefer to have no fees on the card be it secondary or primary card
7. Nearly 20% of customers have changed their provider in last month
8. More than 70% of customers have not defaulted in any loan 
9. Most of the customer have "Single type" family
10. 30% of the customer belong to "Sales & office" job category
11. 50% of the customer are not married
12. Nearly 80% of the customer have not stated their reason for being a customer
13. 90% of the customers have not responded to any of the product offered to them
14. 90% of the customer dont belong to any union and are not retired

#### Bivariate Analysis of Categorical features with the target varibale

In [ ]:
bivaritae_cat_analysis = Data.loc[:,(Data.columns.difference(['age','ed',"employ",'income','debtinc','creddebt','othdebt','spoused','reside','pets','pets_cats','pets_dogs','pets_birds','pets_reptiles','pets_small',
                      'pets_saltfish','pets_freshfish','address','cars','carvalue','commutetime','cardtenure','card2tenure','tenure','longmon',
                      'longten','tollmon','tollten','equipmon','equipten','cardmon','cardten','wiremon','wireten','hourstv']))]

In [ ]:
#for i in bivaritae_cat_analysis.columns:
#    sns.boxplot(x=bivaritae_cat_analysis[i],y="Total_Spend",data=bivaritae_cat_analysis)
#    plt.show()

## Bivariate Analysis of Categorical Variables
1. Out of Card (1,2,3,4,5), Card 1 customers have more credit limit then other card type
2. Out of Card2 (1,2,3,4,5), Card 1 & 4 have more credit limit then other card type
3. People who lease Car have low credit limit as compare to those who own the car
4. Male have higher credit limit then female
5. People who own home have higher total_spend then people who live in rented house
6. People who are in Service of Job Category have more credit limit then other job categories
7. Customers who are "Somewhat Satisfied" With their job have higher total spend
8. Customers who owns' OwnCd, OwnDvd, OwnPda, OwnTv, OwnVcr have higher total_spend
9. People who are customers of bank becuase of "Convenience" have much higher total_spend as comapre to other customers
10. Customers who are not retired are given more total_spend limit

#### Seprating Ordinal and nominal categorical features for one hot encoding/dummy creation of the variable

In [ ]:
Data_cat_nom = Data_cat.loc[:,(Data_cat.columns.difference(["townsize","jobsat","hometype","polview","cardtype","card2type","internet"]))]

In [ ]:
Data_cat_nom.head()

In [ ]:
Data_cat.head(2)

In [ ]:
def create_dummies( df, colname ):
    col_dummies = pd.get_dummies(df[colname], prefix=colname, drop_first=True)
    df = pd.concat([df, col_dummies], axis=1)
    df.drop( colname, axis = 1, inplace = True )
    return df

for c_feature in Data_cat_nom.columns:
    Data_cat[c_feature] = Data_cat[c_feature].astype('category')
    Data_cat = create_dummies(Data_cat , c_feature )

In [ ]:
Data_cat.shape

In [ ]:
Final_data = pd.concat([Data_num,Data_cat],axis =1)

In [ ]:
Final_data.head(2)

In [ ]:
Final_data.shape

In [ ]:
Pre_Final_Data = Final_data.loc[:,Final_data.columns.difference(['Total_Spend'])]

In [ ]:
Pre_Final_Data.head(1)

### Normalizing the data for prediction purpose

In [ ]:
Normal_Data = ((Pre_Final_Data-Pre_Final_Data.min())/(Pre_Final_Data.max()-Pre_Final_Data.min()))

In [ ]:
Normal_Data['Total_Spend'] = Final_data.Total_Spend

In [ ]:
Normal_Data.head(3)

# RFE Method

In [ ]:
feature = Normal_Data.loc[:,(Normal_Data.columns.difference(['Total_Spend']))]

In [ ]:
target = Normal_Data.Total_Spend

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
lm = LinearRegression()

In [ ]:
rfe = RFE(lm,n_features_to_select=30)

In [ ]:
rfe = rfe.fit(feature,target)

In [ ]:
RFE_features = feature.columns[rfe.get_support()]

In [ ]:
feature1 = feature[RFE_features]

In [ ]:
print("Variables Selected from RFE:",feature1.columns)

## Select K best

In [ ]:
X1 = Normal_Data.loc[:,Normal_Data.columns.difference(['Total_Spend'])]

In [ ]:
Y1 = Normal_Data.Total_Spend

In [ ]:
skb1 = SelectKBest(f_regression,k= 30).fit(X1,Y1)

In [ ]:
X1.columns[skb1.get_support()]

## Seprating Normalized Numerical and Categorical Features

In [ ]:
Normal_Num_Data = Normal_Data.loc[:,["age","ed","employ","income","debtinc","creddebt","othdebt","spoused","reside","pets","pets_cats","pets_dogs","pets_birds","pets_reptiles","pets_small","pets_saltfish","pets_freshfish",
                                  "address","cars","carvalue","commutetime","cardtenure","card2tenure","tenure","longmon",
                                    "longten","tollmon","tollten","equipmon","equipten","cardmon","cardten","wiremon","wireten","hourstv","Total_Spend"]]

In [ ]:
Normal_Cat_Data = Normal_Data.loc[:,(Normal_Data.columns.difference(["age","ed","employ","income","debtinc","creddebt","othdebt","spoused","reside","pets","pets_cats","pets_dogs","pets_birds","pets_reptiles","pets_small","pets_saltfish","pets_freshfish",
                                  "address","cars","carvalue","commutetime","cardtenure","card2tenure","tenure","longmon",
                                    "longten","tollmon","tollten","equipmon","equipten","cardmon","cardten","wiremon","wireten","hourstv","Total_Spend"]))]

In [ ]:
Normal_Cat_Data['Total_Spend'] = Normal_Data.Total_Spend

In [ ]:
Normal_Cat_Data.head(2)

#### Correaltion Matrix between independent and dependent variables to see the influence of numerical x variable over y variable

In [ ]:
Correlation = Normal_Num_Data.corr()

In [ ]:
#Correlation.to_excel("Correlation1.xlsx")

#### Selected Variables : ed, income, creddebt,othdebt,carvalue,wireten

#### One Variable-Regression for Categorical and Target(Continous) Variable, selected the variables which have p_value less than 0.05

In [ ]:
f_pvalue = []
fvalue = []
for col in Normal_Cat_Data.columns[0:len(Normal_Cat_Data.columns)-1]:
    model = ""
    formula = 'Total_Spend ~ '+col
    model = smf.ols(formula, data = Normal_Cat_Data).fit()
    f_pvalue.append(model.f_pvalue)
    fvalue.append(model.fvalue)
f_pvalue = pd.Series(f_pvalue, name = "p_value")
fvalue = pd.Series(fvalue, name = "F_score")

In [ ]:
cols = Normal_Cat_Data.columns[0:len(Normal_Cat_Data.columns)-1]
Annova = pd.concat([pd.Series(cols, name = "Col_Names"),f_pvalue,fvalue],axis=1)

In [ ]:
Annova

In [ ]:
Cat_Selected_Var = Annova.Col_Names[Annova.p_value < 0.05]

In [ ]:
Cat_Selected = Normal_Cat_Data[Cat_Selected_Var]

In [ ]:
Cat_Selected.columns

### New Data Set With the reduced features

In [ ]:
Num_Data = Normal_Num_Data.loc[:,["ed","income","creddebt","othdebt","carvalue","wireten"]]

In [ ]:
New_Final_Data = pd.concat([Num_Data,Cat_Selected],axis =1)

In [ ]:
New_Final_Data['Total_Spend'] = Normal_Data.Total_Spend

### RFE Over Selected Variable from the result of Correlation and F-Regression Variables

In [ ]:
feature_new = New_Final_Data.loc[:,(New_Final_Data.columns.difference(['Total_Spend']))]

In [ ]:
target_new = New_Final_Data.Total_Spend

In [ ]:
rfe_new = RFE(lm,n_features_to_select=30)

In [ ]:
rfe_new = rfe_new.fit(feature_new,target_new)

In [ ]:
RFE_Feature_new= feature_new.columns[rfe_new.get_support()]

In [ ]:
features_new_final = feature_new[RFE_Feature_new]

In [ ]:
print("RFE Over Selected Varibles:",features_new_final.columns)

### Select K Best Over Selected Variable from the result of Correlation and F-Regression Variables

In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression

In [ ]:
X = feature_new
Y = target_new

In [ ]:
skb = SelectKBest(f_regression,k = 30).fit(X,Y)

In [ ]:
X.columns[skb.get_support()]

# PCA

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
sc = StandardScaler()

In [ ]:
PCA_DATA = Final_data.loc[:,Final_data.columns.difference(['Total_Spend'])]

In [ ]:
Stat_model = sc.fit(PCA_DATA)

In [ ]:
Transform_data = Stat_model.transform(PCA_DATA)

In [ ]:
Transform_data = pd.DataFrame(Transform_data)

In [ ]:
Transform_data.columns = PCA_DATA.columns
Transform_data

In [ ]:
pca = PCA(n_components=30, random_state=123)

In [ ]:
pca_model = pca.fit(Transform_data)

In [ ]:
pca_model.explained_variance_

In [ ]:
np.cumsum(pca_model.explained_variance_ratio_)

In [ ]:
telco_data = pd.DataFrame(pca.transform(Transform_data),columns=['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12','PC13','PC14','PC15','PC16',
                                                                'PC17','PC18','PC19','PC20','PC21','PC22','PC23','PC24','PC25','PC26','PC27','PC28','PC29','PC30'
                                                                ])

In [ ]:
telco_data

In [ ]:
Loadings =  pd.DataFrame((pca_model.components_.T * np.sqrt(pca_model.explained_variance_)).T,columns=Transform_data.columns).T

In [ ]:
Loadings

In [ ]:
#Loadings.to_excel("PCA.xlsx")

In [ ]:
PCA_Variables = ["card2tenure","cardtenure","tenure","wireless_1","wiremon","equipmon","tollfree_1","carvalue",
                "marital_1","commutecar_1","townsize","pets","carbought_1","card_2","cartype_1","card_4",
                "reason_9","commute_4","commute_8","commute_3","commute_10","card2benefit_3","commute_2",
                "card2benefit_4"]

In [ ]:
print("PCA VARIABLES :",PCA_Variables) 

In [ ]:
print(feature1.columns) #RFE_ORIGINAL

In [ ]:
print(features_new_final.columns) #RFE_REDUCED_FROM_FREGRESSION & CORRELATION

In [ ]:
print(X.columns[skb.get_support()]) #KBest on Reduced variables from F-Regresssion and Correlation

In [ ]:
X1.columns[skb1.get_support()] #K best on the entire data

### Final Selected variabels from the conclsion of multiple varible reduction methods

In [ ]:
Selected_Variables= Normal_Data.loc[:,["card2_2","card2_3","card2_4","card2_5","card_2","card_3","card_4","card_5","carown_0","carown_1","carvalue","creddebt","othdebt","ed","income","gender_1","wireten","internet","owncd_1","owndvd_1","owntv_1","ownvcr_1","reason_2","reason_4","retire_1"]]

In [ ]:
Selected_Variables

### Performing VIF to check the Multicolinearity among the selected vairbales and further removing the vairbels one by one which have VIF score more than 5

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
#the independent variables set 
X = Selected_Variables 
  
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                          for i in range(len(X.columns))] 
  
print(vif_data)

#### Variables with high VIF are carown_1,carvalue,ed,owncd_1,owndvd_1,owntv_1,ownvcr_1. We will remove them one by one

In [ ]:
Selected_Variables= Selected_Variables.loc[:,Selected_Variables.columns.difference(["carown_1","carvalue","ed","owncd_1","owndvd_1","owntv_1","ownvcr_1","ownvcr_1"])]

In [ ]:
Selected_Variables['Total_Spend'] = Normal_Data.Total_Spend

In [ ]:
Selected_Variables

In [ ]:
"+".join(Selected_Variables.columns)

### Splitting the data into test and train for Model building

In [ ]:
train, test = train_test_split(Selected_Variables,test_size = 0.3,random_state=123)

In [ ]:
model = smf.ols(formula="Total_Spend~card2_2+card2_3+card2_4+card2_5+card_2+card_3+card_4+card_5+carown_0+gender_1+income+reason_2+retire_1",data=train).fit()

In [ ]:
print(model.summary())

### Mathematical Equation

In [ ]:
print("Y = 6.6855-0.4036*card2_2-0.3763*card2_3-0.4255*card2_4-0.2967*card2_5-0.5860*card_2-0.6073*card_3-0.7080*card_4-0.5245*card_5-0.0669*carown_0-0.0464*gender_1+1.1351*income+0.2671*reason_2-0.2065*retire_1")

# Predicting on Training and metric calculation

In [ ]:
train['pred'] = np.exp(model.predict(train))
train['Total_Spend'] = np.exp(train.Total_Spend)

In [ ]:
error_train = train.Total_Spend - train.pred
abs_error_train = abs(error_train)
pct_abs_error_train = abs_error_train/train.Total_Spend
MAPE_train = pct_abs_error_train.mean()
print("MAPE TRAIN:",MAPE_train)

# Predicting on Test and Metric Calculations

In [ ]:
test['pred'] = np.exp(model.predict(test))
test['Total_Spend'] = np.exp(test.Total_Spend)

In [ ]:
error_test = test.Total_Spend - test.pred
abs_error_test = abs(error_test)
pct_abs_error_test = abs_error_test/test.Total_Spend
Mape_test = pct_abs_error_test.mean()
print("MAPE TEST:",Mape_test)

In [ ]:
RMSE_Train_ols= metrics.mean_squared_error(train.pred,train.Total_Spend)**(1/2)
RMSE_Test_ols= metrics.mean_squared_error(test.pred,test.Total_Spend)**(1/2)

In [ ]:
ols = {"R_Square_ols":[model.rsquared],"Mape_Train_ols":MAPE_train,"Mape_Test_ols":Mape_test,"RMSE_Train_ols":RMSE_Train_ols,"RMSE_Test_ols":RMSE_Test_ols}

In [ ]:
ols = pd.DataFrame.from_dict(ols)

In [ ]:
ols.T

### Checking the residual graph, it should follow normal distribution

In [ ]:
sns.distplot(model.resid)
plt.show()

### Checking the correlation between actual and error, actual and predcited

In [ ]:
import scipy.stats as stats

In [ ]:
print("Correlation Between Actucal and Error on Train:",stats.stats.pearsonr(y=train.Total_Spend,x=error_train))
print("Correlation Between Actucal and Predicted on Train:",stats.stats.pearsonr(y=train.Total_Spend,x=train.pred))

In [ ]:
print("Correlation Between Actucal and Error on Test:",stats.stats.pearsonr(y=test.Total_Spend,x=error_test))
print("Correlation Between Actucal and Predicted on Test:",stats.stats.pearsonr(y=test.Total_Spend,x=test.pred))

# Decile Analysis

In [ ]:
train['Deciles']=pd.qcut(train['pred'],10, labels=False)
Predicted_avg = train[['Deciles','pred']].groupby(train.Deciles).mean().sort_index(ascending=False)['pred']
Actual_avg = train[['Deciles','Total_Spend']].groupby(train.Deciles).mean().sort_index(ascending=False)['Total_Spend']

Decile_analysis_train = pd.concat([Predicted_avg, Actual_avg], axis=1)

Decile_analysis_train

In [ ]:
Decile_analysis_train.to_excel("Decile_Train.xlsx")

In [ ]:
test['Deciles']=pd.qcut(test['pred'],10, labels=False)
Predicted_avg = test[['Deciles','pred']].groupby(test.Deciles).mean().sort_index(ascending=False)['pred']
Actual_avg = test[['Deciles','Total_Spend']].groupby(test.Deciles).mean().sort_index(ascending=False)['Total_Spend']

Decile_analysis_test = pd.concat([Predicted_avg, Actual_avg], axis=1)

Decile_analysis_test

In [ ]:
Decile_analysis_test.to_excel("Decile_Test.xlsx")

# Splitting Data for Machine Learning

In [ ]:
Y =np.exp(Final_data.Total_Spend)

In [ ]:
X = Pre_Final_Data.loc[:,["card2_2","card2_3","card2_4","card2_5","card_2","card_3","card_4","card_5","carown_0","carown_1","carvalue","creddebt","ed","gender_1","income","internet","othdebt","owncd_1","owndvd_1","ownvcr_1","reason_2","reason_4","retire_1","wireten"]]

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X,Y,test_size = 0.3,random_state= 345)

# KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
KNN_Data = sc.fit_transform(X)
KNN_Data = pd.DataFrame(KNN_Data)
KNN_Data.columns = X.columns

In [ ]:
train_X1, test_x1, train_y1, test_y1 = train_test_split(KNN_Data,Y,test_size=0.3, random_state = 888)

In [ ]:
param_grid_knn = [{"n_neighbors" : [3,5,7,9,11,13], "leaf_size" : [20,30,40,50,60]}]

In [ ]:
model_knn = GridSearchCV(KNeighborsRegressor(),param_grid_knn,cv=3,scoring="r2")
knn_model= model_knn.fit(train_X1,train_y1)

In [ ]:
print(knn_model.best_params_)
print(knn_model.best_score_)

In [ ]:
best_model_knn = KNeighborsRegressor(n_neighbors=13,leaf_size=20)
best_model_knn.fit(train_X1,train_y1)

In [ ]:
train_r2 = metrics.r2_score(train_y1,best_model_knn.predict(train_X1))
print("R-Square Train:",train_r2)

In [ ]:
best_model_knn.fit(test_x1,test_y1)

In [ ]:
print("Mean Absolute Error KNN Train:",metrics.mean_absolute_error(train_y1,best_model_knn.predict(train_X1)))
print("Mean Absolute Error KNN Test:",metrics.mean_absolute_error(test_y1,best_model_knn.predict(test_x1)))
print("Mean Square Error KNN Train:",metrics.mean_squared_error(train_y1,best_model_knn.predict(train_X1)))
print("Mean Square Error KNN Test:",metrics.mean_squared_error(test_y1,best_model_knn.predict(test_x1)))

In [ ]:
error_knn = train_y1-best_model_knn.predict(train_X1)
abs_error_knn = abs(error_knn)
abs_pct_error_knn = abs_error_knn/train_y1
MAPE_knn = abs_pct_error_knn.mean()
print("MAPE KNN Train:",MAPE_knn)

error_knn_test = test_y1-best_model_knn.predict(test_x1)
abs_error_knn_test = abs(error_knn_test)
abs_pct_error_knn_test = abs_error_knn_test/test_y1
MAPE_knn_test = abs_pct_error_knn_test.mean()
print("MAPE KNN Test:",MAPE_knn_test)

print("RMSE KNN Train:",metrics.mean_squared_error(train_y1,best_model_knn.predict(train_X1))**(1/2))
print("RMSE KNN Test:",metrics.mean_squared_error(test_y1,best_model_knn.predict(test_x1))**(1/2))

In [ ]:
knn = {"R_Square_knn":train_r2,"Mape_Train_knn":MAPE_knn,"Mape_Test_knn":MAPE_knn_test,
       "RMSE_Train_knn":[metrics.mean_squared_error(train_y1,best_model_knn.predict(train_X1))**(1/2)],
      "RMSE_Test_knn":[metrics.mean_squared_error(test_y1,best_model_knn.predict(test_x1))**(1/2)]}
knn = pd.DataFrame.from_dict(knn)
knn

# DT Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
param_grid_dtr = {"max_depth":np.arange(2,5),"max_features": np.arange(4,7),"random_state":[100]}

In [ ]:
model_dtr = GridSearchCV(DecisionTreeRegressor(),param_grid_dtr,cv = 3,scoring="r2")

In [ ]:
model_dtr.fit(train_X,train_y)

In [ ]:
print(model_dtr.best_params_)
print(model_dtr.best_score_)

In [ ]:
best_model_dtr = DecisionTreeRegressor(max_depth=4,max_features=6)
best_model_dtr.fit(train_X,train_y)

In [ ]:
R2_DT_TRAIN = metrics.r2_score(train_y,best_model_dtr.predict(train_X))
R2_DT_TRAIN

In [ ]:
best_model_dtr.fit(test_X,test_y)

In [ ]:
print("Mean Absolute Error Train:",metrics.mean_absolute_error(train_y,best_model_dtr.predict(train_X)))
print("Mean Absolute Error Test:",metrics.mean_absolute_error(test_y,best_model_dtr.predict(test_X)))

print("Mean Square Error DT Train:",metrics.mean_squared_error(train_y,best_model_dtr.predict(train_X)))
print("Mean Square Error DT Test:",metrics.mean_squared_error(test_y,best_model_dtr.predict(test_X)))

error_dt = train_y-best_model_dtr.predict(train_X)
abs_error_dt = abs(error_dt)
abs_pct_error_dt = abs_error_dt/train_y
MAPE_dt = abs_pct_error_dt.mean()
print("MAPE Decision Tree Train:",MAPE_dt)

error_dt_test = test_y-best_model_dtr.predict(test_X)
abs_error_dt_test = abs(error_dt_test)
abs_pct_error_dt_test = abs_error_dt_test/test_y
MAPE_dt_test = abs_pct_error_dt_test.mean()
print("MAPE Decision Tree Test:",MAPE_dt_test)

print("RMSE Decision Tree Train:",metrics.mean_squared_error(train_y,best_model_dtr.predict(train_X))**(1/2))
print("RMSE Decision Tree Test:",metrics.mean_squared_error(test_y,best_model_dtr.predict(test_X))**(1/2))

In [ ]:
DT = {"R_Square_dt":R2_DT_TRAIN,
      "Mape_Train_dt":MAPE_dt,
      "Mape_Test_dt":MAPE_dt_test,
       "RMSE_Train_dt":[metrics.mean_squared_error(train_y,best_model_dtr.predict(train_X))**(1/2)],
      "RMSE_Test_dt":[metrics.mean_squared_error(test_y,best_model_dtr.predict(test_X))**(1/2)]}
DT = pd.DataFrame.from_dict(DT)
DT

In [ ]:
best_model_dtr.feature_importances_

In [ ]:
train_X.columns

In [ ]:
var_imp = pd.DataFrame([train_X.columns, best_model_dtr.feature_importances_]).T
var_imp.columns = ['var', 'imp']
var_imp.sort_values(by= 'imp', inplace=True)
var_imp.plot(kind='barh')

# Ensemble Learnig - Bagging,RandomForset,AdaBoost and GBM

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Bagging

In [ ]:
param_grid_bagg = {"n_estimators":[50,60,70,80,90,100],"random_state":[67]}

In [ ]:
model_bagg = GridSearchCV(BaggingRegressor(),param_grid=param_grid_bagg,cv=3,n_jobs=-1)
model_bagg.fit(train_X,train_y)

In [ ]:
model_bagg.best_params_

In [ ]:
best_model_bagg = BaggingRegressor(n_estimators=100)
best_model_bagg.fit(train_X,train_y)

In [ ]:
R_Sqaure_Bagging= metrics.r2_score(train_y,best_model_bagg.predict(train_X))
R_Sqaure_Bagging

In [ ]:
best_model_bagg.fit(test_X,test_y)

In [ ]:
print("Mean Absolute Error Train:",metrics.mean_absolute_error(train_y,best_model_bagg.predict(train_X)))
print("Mean Absolute Error Test:",metrics.mean_absolute_error(test_y,best_model_bagg.predict(test_X)))

In [ ]:
print("Mean Square Error Bagging Train:",metrics.mean_squared_error(train_y,best_model_bagg.predict(train_X)))
print("Mean Square Error Bagging Test:",metrics.mean_squared_error(test_y,best_model_bagg.predict(test_X)))

In [ ]:
error_Bagg = train_y-best_model_bagg.predict(train_X)
abs_error_Bagg = abs(error_Bagg)
abs_pct_error_bagg = abs_error_Bagg/train_y
MAPE_bagg = abs_pct_error_bagg.mean()
print("MAPE Bagging Train:",MAPE_bagg)

error_bagg_test = test_y-best_model_bagg.predict(test_X)
abs_error_bagg_test = abs(error_bagg_test)
abs_pct_error_bagg_test = abs_error_bagg_test/test_y
MAPE_bagg_test = abs_pct_error_bagg_test.mean()
print("MAPE Bagging Test:",MAPE_bagg_test)

In [ ]:
print("RMSE Bagging Train:",metrics.mean_squared_error(train_y,best_model_bagg.predict(train_X))**(1/2))
print("RMSE Bagging Test:",metrics.mean_squared_error(test_y,best_model_bagg.predict(test_X))**(1/2))

In [ ]:
Bagging = {"R_Square_bagging":R_Sqaure_Bagging,
      "Mape_Train_bagging":MAPE_bagg,
      "Mape_Test_bagging":MAPE_bagg_test,
       "RMSE_Train_bagging":[metrics.mean_squared_error(train_y,best_model_bagg.predict(train_X))**(1/2)],
      "RMSE_Test_bagging":[metrics.mean_squared_error(test_y,best_model_bagg.predict(test_X))**(1/2)]}
Bagging = pd.DataFrame.from_dict(Bagging)
Bagging

# Random Forest

In [ ]:
param_grid_rf = {"n_estimators":[50,60,70,80,90,100],"max_depth":[2,3,4],"random_state":[89]}
model_rf = GridSearchCV(RandomForestRegressor(),param_grid=param_grid_rf,cv = 3, n_jobs=-1)
model_rf.fit(train_X,train_y)

In [ ]:
print(model_rf.best_params_)
print(model_rf.best_score_)
print(model_rf.cv)

In [ ]:
best_model_rf = RandomForestRegressor(n_estimators=50,max_depth=4)
result_rf = best_model_rf.fit(train_X,train_y)

In [ ]:
R2_RF_Train= metrics.r2_score(train_y,result_rf.predict(train_X))
R2_RF_Train

In [ ]:
result_rf_test = best_model_rf.fit(test_X,test_y)

In [ ]:
print("Mean Absolute Error Train:",metrics.mean_absolute_error(train_y,result_rf.predict(train_X)))
print("Mean Absolute Error Test:",metrics.mean_absolute_error(test_y,result_rf_test.predict(test_X)))

In [ ]:
print("Mean Square Error RF Train:",metrics.mean_squared_error(train_y,result_rf.predict(train_X)))
print("Mean Square Error RF Test:",metrics.mean_squared_error(test_y,result_rf_test.predict(test_X)))
print("RMSE RF Train:",metrics.mean_squared_error(train_y,result_rf.predict(train_X))**(1/2))
print("RMSE RF Test:",metrics.mean_squared_error(test_y,result_rf_test.predict(test_X))**(1/2))

In [ ]:
error_rf_train = train_y-result_rf.predict(train_X)
abs_error_rf_train = abs(error_rf_train)
abs_pct_error_rf_train = abs_error_rf_train/train_y
MAPE_rf_train = abs_pct_error_rf_train.mean()
print("MAPE RF Train:",MAPE_rf_train)


error_rf_test = test_y-result_rf_test.predict(test_X)
abs_error_rf_test = abs(error_rf_test)
abs_pct_error_rf_test = abs_error_rf_test/test_y
MAPE_rf_test = abs_pct_error_rf_test.mean()
print("MAPE RF Test:",MAPE_rf_test)

In [ ]:
Random_Forest = {"R_Square_rf":R2_RF_Train,
      "Mape_Train_rf":MAPE_rf_train,
      "Mape_Test_rf":MAPE_rf_test,
       "RMSE_Train_rf":[metrics.mean_squared_error(train_y,result_rf.predict(train_X))**(1/2)],
      "RMSE_Test_rf":[metrics.mean_squared_error(test_y,result_rf_test.predict(test_X))**(1/2)]}
Random_Forest = pd.DataFrame.from_dict(Random_Forest)
Random_Forest

In [ ]:
stats.stats.pearsonr(train_y,error_rf_train)

In [ ]:
stats.stats.pearsonr(train_y,result_rf.predict(train_X))

In [ ]:
stats.stats.pearsonr(test_y,error_rf_test)

In [ ]:
stats.stats.pearsonr(test_y,result_rf_test.predict(test_X))

In [ ]:
result_rf.feature_importances_

In [ ]:
train_X.columns

In [ ]:
result_rf.feature_importances_
indices = np.argsort(result_rf.feature_importances_)[::-1]

In [ ]:
indices = np.argsort(result_rf.feature_importances_)[::-1]
feature_rank = pd.DataFrame( columns = ['rank', 'feature', 'importance'] )
for f in range(train_X.shape[1]):
  feature_rank.loc[f] = [f+1,
                         train_X.columns[indices[f]],
                         result_rf.feature_importances_[indices[f]]]
sns.barplot( y = 'feature', x = 'importance', data = feature_rank )
plt.show()

# AdaBoost

In [ ]:
param_grid_ada = {"n_estimators":[60,70,80,90,100],"learning_rate":[10**x for x in range(-4,2)]}
model_ada = GridSearchCV(AdaBoostRegressor(),param_grid=param_grid_ada,cv=3,verbose=True,n_jobs=-1)
result_ada = model_ada.fit(train_X,train_y)

In [ ]:
print(result_ada.best_params_)
print(result_ada.best_score_)

In [ ]:
best_model_ada = AdaBoostRegressor(n_estimators=60,learning_rate=0.1)
result_ada = best_model_ada.fit(train_X,train_y)

In [ ]:
result_ada.predict(train_X)

In [ ]:
R2_Ada_Train = metrics.r2_score(train_y,result_ada.predict(train_X))
R2_Ada_Train

In [ ]:
result_ada_test = best_model_ada.fit(test_X,test_y)

In [ ]:
print("Mean Absolute Error Train:",metrics.mean_absolute_error(train_y,result_ada.predict(train_X)))
print("Mean Absolute Error Test:",metrics.mean_absolute_error(test_y,result_ada_test.predict(test_X)))

In [ ]:
print("Mean Square Error RF Train:",metrics.mean_squared_error(train_y,result_ada.predict(train_X)))
print("Mean Square Error RF Test:",metrics.mean_squared_error(test_y,result_ada_test.predict(test_X)))
print("RMSE RF Train:",metrics.mean_squared_error(train_y,result_ada.predict(train_X))**(1/2))
print("RMSE RF Test:",metrics.mean_squared_error(test_y,result_ada_test.predict(test_X))**(1/2))

In [ ]:
error_ada_train = train_y-result_ada.predict(train_X)
abs_error_ada_train = abs(error_ada_train)
abs_pct_error_ada_train = abs_error_ada_train/train_y
MAPE_ada_train = abs_pct_error_ada_train.mean()
print("MAPE ADA Train:",MAPE_ada_train)


error_ada_test = test_y-result_ada_test.predict(test_X)
abs_error_ada_test = abs(error_ada_test)
abs_pct_error_ada_test = abs_error_ada_test/test_y
MAPE_ada_test = abs_pct_error_ada_test.mean()
print("MAPE ADA Test:",MAPE_ada_test)

In [ ]:
Ada_Boost = {"R_Square_ada":R2_Ada_Train,
      "Mape_Train_ada":MAPE_ada_train,
      "Mape_Test_ada":MAPE_ada_test,
       "RMSE_Train_ada":[metrics.mean_squared_error(train_y,result_ada.predict(train_X))**(1/2)],
      "RMSE_Test_ada":[metrics.mean_squared_error(test_y,result_ada_test.predict(test_X))**(1/2)]}
Ada_Boost = pd.DataFrame.from_dict(Ada_Boost)
Ada_Boost

### Gradient Boost

In [ ]:
param_grid_gbm = {"n_estimators":[60,70,80,90,100],
                 "max_depth":[2,3,4,5],
                 "learning_rate":[10**x for x in range(-4,2)],
                 "random_state":[56]}
model_gbm = GridSearchCV(GradientBoostingRegressor(),param_grid=param_grid_gbm,cv = 3, verbose=True,n_jobs=-1)
result_gbm = model_gbm.fit(train_X,train_y)

In [ ]:
print("Best Parameters:",result_gbm.best_params_)
print("Best Scores:",result_gbm.best_score_)

In [ ]:
best_model_gbm = GradientBoostingRegressor(n_estimators=100,max_depth=3,learning_rate=0.1)
result_gbm = best_model_gbm.fit(train_X,train_y)

In [ ]:
R2_GBM_Train = metrics.r2_score(train_y,result_gbm.predict(train_X))
R2_GBM_Train

In [ ]:
result_gbm_test = best_model_gbm.fit(test_X,test_y)

In [ ]:
print("MAE GBM TRAIN:",metrics.mean_absolute_error(train_y,result_gbm.predict(train_X)))
print("MAE GBM TEST:",metrics.mean_absolute_error(test_y,result_gbm_test.predict(test_X)))

In [ ]:
print("MSE GBM TRAIN:",metrics.mean_squared_error(train_y,result_gbm.predict(train_X)))
print("MSE GBM TEST:",metrics.mean_squared_error(test_y,result_gbm_test.predict(test_X)))
print("RMSE GBM TRAIN:",metrics.mean_squared_error(train_y,result_gbm.predict(train_X))**(1/2))
print("RMSE GBM TEST:",metrics.mean_squared_error(test_y,result_gbm_test.predict(test_X))**(1/2))

In [ ]:
error_gbm_train = train_y - result_gbm.predict(train_X)
abs_error_gbm_train = abs(error_gbm_train)
abs_pct_error_gbm_train  = abs_error_gbm_train/train_y
mape_gbm_train = abs_pct_error_gbm_train.mean()
print("MAPE GBM TRAIN:",mape_gbm_train)


error_gbm_test = test_y - result_gbm_test.predict(test_X)
abs_error_gbm_test = abs(error_gbm_test)
abs_pct_error_gbm_test  = abs_error_gbm_test/test_y
mape_gbm_test = abs_pct_error_gbm_test.mean()
print("MAPE GBM TREST:",mape_gbm_test)

In [ ]:
GBM = {"R_Square_GBM":R2_GBM_Train,
      "Mape_Train_GBM":mape_gbm_train,
      "Mape_Test_GBM":mape_gbm_test,
       "RMSE_Train_GBM":[metrics.mean_squared_error(train_y,result_gbm.predict(train_X))**(1/2)],
      "RMSE_Test_GBM":[metrics.mean_squared_error(test_y,result_gbm_test.predict(test_X))**(1/2)]}
GBM = pd.DataFrame.from_dict(GBM)
GBM

### XGBOOST

In [ ]:
import xgboost

In [ ]:
param_grid_xg = {"n_estimators":[60,70,80,90,100,120],
                "max_depth":[2,3,4,5]}
model_xg = GridSearchCV(xgboost.XGBRegressor(),param_grid=param_grid_xg,cv=3,n_jobs=-1)
result_xg = model_xg.fit(train_X,train_y)

In [ ]:
print("Best Parmaeters:",result_xg.best_params_)
print("Best Score:",result_xg.best_score_)

In [ ]:
best_model_xg = xgboost.XGBRegressor(n_estimators = 70,max_depth=2)
result_xg = best_model_xg.fit(train_X,train_y)

In [ ]:
R2_XG_Train=metrics.r2_score(train_y,result_xg.predict(train_X))
R2_XG_Train

In [ ]:
result_xg_test = best_model_xg.fit(test_X,test_y)

In [ ]:
print("RMSE XG TRAIN:",metrics.mean_squared_error(train_y,result_xg.predict(train_X))**(1/2))
print("RMSE XG TEST:",metrics.mean_squared_error(test_y,result_xg_test.predict(test_X))**(1/2))

In [ ]:
error_xg_train = train_y - result_xg.predict(train_X)
abs_error_xg_train = abs(error_xg_train)
abs_pct_error_xg_train  = abs_error_xg_train/train_y
mape_xg_train = abs_pct_error_xg_train.mean()
print("MAPE XG TRAIN:",mape_xg_train)


error_xg_test = test_y - result_xg_test.predict(test_X)
abs_error_xg_test = abs(error_xg_test)
abs_pct_error_xg_test  = abs_error_xg_test/test_y
mape_xg_test = abs_pct_error_xg_test.mean()
print("MAPE XG TEST:",mape_xg_test)

In [ ]:
XGboost = {"R_Square_xg":R2_XG_Train,
      "Mape_Train_xg":mape_xg_train,
      "Mape_Test_xg":mape_xg_test,
       "RMSE_Train_xg":[metrics.mean_squared_error(train_y,result_xg.predict(train_X))**(1/2)],
      "RMSE_Test_xg":[metrics.mean_squared_error(test_y,result_xg_test.predict(test_X))**(1/2)]}
XGboost = pd.DataFrame.from_dict(XGboost)
XGboost

In [ ]:
result_xg.feature_importances_

In [ ]:
train_X.columns

In [ ]:
result_xg.feature_importances_
indices = np.argsort(result_xg.feature_importances_)[::-1]

In [ ]:
indices = np.argsort(result_xg.feature_importances_)[::-1]
feature_rank_xg = pd.DataFrame( columns = ['rank', 'feature', 'importance'] )
for f in range(train_X.shape[1]):
  feature_rank_xg.loc[f] = [f+1,
                         train_X.columns[indices[f]],
                         result_xg.feature_importances_[indices[f]]]
sns.barplot( y = 'feature', x = 'importance', data = feature_rank )
plt.show()

In [ ]:
print(ols)
print(knn)
print(DT)
print(Bagging)
print(Random_Forest)
print(Ada_Boost)
print(GBM)
print(XGboost)